<a href="https://colab.research.google.com/github/dope232/transformers_application-/blob/main/mytransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Remove non alphanumeric characters for simple training
- Sentence tokenization


In [ ]:

import torch
import numpy as np

In [ ]:
import importlib.util
import sys


file_path = '/kaggle/input/transformer/transformers4.py'

spec = importlib.util.spec_from_file_location("transformers4", file_path)
transformers4 = importlib.util.module_from_spec(spec)
sys.modules["transformers4"] = transformers4
spec.loader.exec_module(transformers4)


In [ ]:
if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("GPU is not available.")

In [ ]:
english_file = '/kaggle/input/language/train.en'
hindi_file = '/kaggle/input/language/train.hi'



START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

hindi_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                    'ँ', 'ं', 'ः', 'ऄ', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ऍ', 'ऎ', 'ए', 'ऐ',
                    'ऑ', 'ऒ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ',
                    'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'ऩ', 'प', 'फ', 'ब', 'भ',
                    'म', 'य', 'र', 'ऱ', 'ल', 'ळ', 'ऴ', 'व', 'श', 'ष', 'स', 'ह',
                    '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'ॅ', 'ॆ', 'े', 'ै', 'ॉ', 'ॊ',
                    'ो', 'ौ', '्', 'ॎ', 'ॏ', 'ॐ', '॑', '॒', '॓', '॔', 'ॕ', 'ॖ', 'ॗ', 'क़', 'ख़',
                    'ग़', 'ज़', 'ड़', 'ढ़', 'फ़', 'य़', 'ॠ', 'ॡ', 'ॢ', 'ॣ', '।', '॥',
                    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', PADDING_TOKEN, END_TOKEN]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [ ]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_hindi = {k:v for k,v in enumerate(english_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [ ]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(hindi_file, 'r') as file:
    hindi_sentences = file.readlines()


TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

In [ ]:
english_sentences[:10]

["however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles",
 'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.',
 'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.',
 'mithali to anchor indian team against australia in odis',
 'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence',
 'the court has fixed a hearing for february 12',
 'please select the position where the track should be split.',
 'as per police, armys 22rr, special operation group (sog) of police and the central reserve police force (crpf) cordoned the village and launched search operation in the area.',
 'jharkhand chief minister hemant soren',
 'arvind kumar, sho of the sector 55/56 police sta

In [ ]:
hindi_sentences[:10]

['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।',
 'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
 '8 सितम्\u200dबर, 2016 को माननीय राष्\u200dट्रपति की स्\u200dवीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्\u200dव में आया',
 'अदालत ने इस मामले में आगे की सुनवाई के लिए एक फरवरी की तारीख़ तय की',
 'जहाँ पर ट्रैक को विभाजित किया जाना है, कृपया वह स्थान चुनें.',
 'इसके तुरंत बाद सेना की 22 राष्ट्रीय राइफल्स (आरआर), सीआरपीएफ और पुलिस के स्पेशल ऑपरेशन ग्रुप (एसओजी) के जवानों द्वारा इलाके की घेराबंदी कर तलाशी अभियान चलाया।',
 'झारखंड के मुख्यमंत्री हेमंत सोरेन (फोटोः पीटीआई)',
 'सेक्टर 55/56

In [ ]:
max_sequence_length = 250

def tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1)

valid_sentence_indicies = []
for i in range(len(hindi_sentences)):
    hindi_sentences, english_sentence = hindi_sentences[i], english_sentences[i]
    if valid_length(hindi_sentences, max_sequence_length) \
      and valid_length(english_sentence, max_sequence_length) \
      and tokens(hindi_sentences, hindi_vocabulary):
        valid_sentence_indicies.append(i)



Number of sentences: 200000
Number of valid sentences: 156465


In [ ]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [ ]:
hindi_sentences[:3]

['और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।',
 'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को']

In [ ]:
import torch

dmodel = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 250
hin_vocab_size = len(hindi__vocabulary)

transformer = transformers4.Transformer(dmodel,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          hin_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [ ]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, i):
        return self.english_sentences[i], self.hindi_sentences[i]

In [ ]:
dataset = TextDataset(english_sentences, hindi_sentences)

In [ ]:
len(dataset)

156465

In [ ]:
dataset[1]

('the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.',
 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।')

In [ ]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.', 'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.', 'mithali to anchor indian team against australia in odis', 'the court has fixed a hearing for february 12', 'please select the position where the track should be split.', 'as per police, armys 22rr, special operation group (sog) of police and the central reserve police force (crpf) cordoned the village and launched search operation in the area.', 'jharkhand chief minister hemant soren', 'arvind kumar, sho of the sector 55/56 police station, said a case has been registered under section 376-d (gang rape) of the indian penal code.', "briefing media in new delhi today, party's state-in-charge, anil jain said, after the meeting the party will meet the governor to stake claim for government formation in the state.", '

In [ ]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')


for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hin_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for i in range(num_sentences):
      eng_sentence_length, hin_sentence_length = len(eng_batch[i]), len(hin_batch[i])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      hin_chars_to_padding_mask = np.arange(hin_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[i, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[i, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[i, :, hin_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[i, hin_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[i, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[i, hin_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epochs {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hin_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hin_batch)
        optim.zero_grad()
        hin_predictions = transformer(eng_batch,
                                     hin_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hin_batch, start_token=False, end_token=True)
        loss = criterian(
            hin_predictions.view(-1, hin_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()

        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hin_batch[0]}")
            hin_sentence_predicted = torch.argmax(hin_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hin_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hin_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hin_sentence)
                predictions = transformer(eng_sentence,
                                          hin_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hin_sentence = (hin_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {hin_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 6.3858160972595215
English: whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.
Hindi Translation: और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है
Hindi Prediction: 5"666ऄॊऄञऊ666ऄऄऄ6ऄऄऄ66ड6ॆ6)"6ञञ6ऄ6ऄऄ66666ऄ66ढ66666ॊॊऱऄऄ666ःॊॊः6ः66666+ऄ5)6ढ6666ऄ66666ऄ6ऄ66ऄऄऄ66ऄ6ऄऄऄऄऄऄऄऄःःऄऄऄभऄऄऄऄऄऄऄऄऄऄऄःःऄऄऄऄःऄःःऄःऄ)+:ःऄऄ5॓ऄ)ऄ॓ऄभ॓ऄऄऄभणणणभणऄऄऄऄऄऄऄ66॓॓॓॓6धॐ॓॓॓॓ऄऄभ5भभ5भॆ5ऄ56ऄ+ऄऄऄऄभऄऄ)भ+++ः+भभऄ6+णणणभण४४ध))(ण(6॓ड़भ॓((णणण॓ञभऄण(=+(भण6ण6
Evaluation translation (should we go to the mall?) : ('              डीीीड  (ीीीीी(((ॊबब  ीीीॊॊॊॊॊॊॊॊ(666ॊॊॊााललॊॊॊॊॊॊॊॊ6666 लललली)))     6 6बब    ःःःॊॊःॊऊऊऊःःःःःःःःः।भऄऄऄऄऄऄऄऄऄःःःःःःःःःःःःःओॖॖॖसस(((((॓॓॓॓भभभभभभभ(॓॓॓॓(((((॓॓॓॓॓॓॓॓॓॓॓॓॓ीीओभ555555555ओऽभभभभभभभभभःःःः(((((॓(((४४४((((((((((((((((((((((((((((((',)
-------------------------------------------
Iteration 10

## Inference

In [ ]:
transformer.eval()
def output(eng_sentence):
  eng_sentence = (eng_sentence,)
  hin_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hin_sentence)
    predictions = transformer(eng_sentence,
                              hin_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hin_sentence = (hin_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hin_sentence[0]

In [ ]:
translation = output("as a result, you will be fired")
print(translation)


जिसके बाद में काम करने के लिए काम करना चाहिए<END>


In [ ]:
translation = output("it is a beautiful day outside")
print(translation)


यह समय का सामना करना चाहिए<END>


In [ ]:
translation = output("birds are singing")
print(translation)


बारिश के बारे में बढ़े में बढ़ा रहे<END>


In [ ]:
translation = output("")
print(translation)


कालें को करें<END>


In [ ]:
translation = output("go to jail")
print(translation)

जेएनयू के साथ जाना बनाए को भी करें<END>


In [ ]:
translation = output("")
print(translation)

बालें में बारे में बारे में<END>


In [ ]:
translation = output("who are you")
print(translation)

क्या है कि वह क्या है?<END>


In [ ]:
translation = output("independent man")
print(translation)


भारतीय करें<END>


In [ ]:
translation = output("politics")
print(translation)


प्रोजित करें<END>


In [ ]:
translation = output("not good")
print(translation)

नहीं कराम कर नहीं कर नहीं कर रही है।<END>


In [ ]:
translation = output("talk together")
print(translation)

कार्यक्रम करें<END>


In [ ]:
translation = output("what should we do?")
print(translation)

क्या हम क्या है?<END>


In [ ]:
translation = output("do this")
print(translation)

इस प्रामक करें हैं।<END>


In [ ]:
translation = output("my my my ")
print(translation)


मेरे में मेरी में मेरी में मेरे में मेरे में मेरे में मेरे में मेरी में मेरी में मेरी में हैं।<END>


In [ ]:
translation = output("let me speak")
print(translation)


कांग्रेस के लिए कार्यकार<END>


In [ ]:
translation = output("i am okay.")
print(translation)


मैं आप में हूं।<END>


In [ ]:
translation = output("i am in your walls")
print(translation)

मैं अपने काम करें<END>
